<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/no_gendered_pronouns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove gendered pronouns.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "no_gen_pronouns"
MODEL_NAME = "no_gen_pronouns"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [24]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [25]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [26]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [27]:
len(train_df)

9879016

In [28]:
len(dev_df)

2292907

In [0]:
# # Get a list of the politicians' names.
# # I am using just the names listed in the op_name series.
# polit_names = train_df['op_name']
# polit_names2 = dev_df['op_name']
# polit_names.drop_duplicates(inplace=True)
# polit_names2.drop_duplicates(inplace=True)

# full_names_list = polit_names.to_list() + polit_names2.to_list()
# polit_names_list = [name for names in full_names_list for name in names.split()]
# # Dedup the list again now that it's split.
# new_list = []
# for name in polit_names_list:
#     if name.lower() not in new_list:
#         new_list.append(name.lower())
# polit_names_list = new_list

In [0]:
# # Remove initials and names that are words with common meaning.
# common_word_list = ['rob','blunt','king','mark','bob','jan','chuck','rod','love','sessions','buddy','mast',
#                     'bill','holding','drew','waters','gene','green','jerry','young','brat','clay','bass',
#                     'rice','jack','hatch','van','buck','tester','dick','g.','brown','fudge','dean','don',
#                     'slaughter','banks','f.','lynch','flake','sires','robin']
# new_list = []
# for name in polit_names_list:
#     if name not in common_word_list:
#         new_list.append(name)
# polit_names_list = new_list

In [29]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [30]:
start_time = time.time()
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
new_train_df['split_response'] = new_train_df['response_text'].str.split()
# Remove the rows with split responses that have length 1 or 2.
train_df = new_train_df[new_train_df['split_response'].str.len() > 2]
train_df.head()
print("seconds to complete:",(time.time()-start_time))
train_df.shape

seconds to complete: 38.13105249404907


(3577352, 9)

In [0]:
# Create a list for simple pronoun substitution.
gen_pronouns = ['she','her','hers','congresswoman','he','his','him','congressman']

In [0]:
# Replace gendered pronouns with simple 'pronoun' token.
from string import punctuation
def replace_pronouns(response_list,gen_pronouns):
    response_list = ' '.join('pronoun' if i.strip(punctuation) in gen_pronouns else i for i in response_list.split())
    return response_list

In [43]:
start_time = time.time()
train_df['response_text'] = train_df['response_text'].astype(str).apply(replace_pronouns,args=(gen_pronouns,))
print("minutes to complete:",(time.time()-start_time)/60)
train_df.head()

minutes to complete: 0.9764049172401428


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican,"[protecting, birth, is, not, the, same, as, protecting, life., you, may, very, well, pledge, to,..."
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican,"[you, need, to, protect, children, and, leave, my, body, to, me.]"
3,3,57265377,M,0,Bob,thank you roger,Roger Williams,Congress_Republican,"[thank, you, roger]"
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican,"[unwanted, pregnancy, is, a, sad, and, unfortunate, situation, for, anyone, to, find, themselves..."
5,5,57265377,M,0,Candice,"women, the ""hosts"" of the unborn babies you are so supportive of, need access to healthcare to g...",Roger Williams,Congress_Republican,"[women,, the, ""hosts"", of, the, unborn, babies, you, are, so, supportive, of,, need, access, to,..."


In [0]:
# # Remove political names from training set.
# from string import punctuation
# def replace_polit_names(response_list,names):
#     response_list = ' '.join('politname' if i.strip(punctuation) in names else i for i in response_list.split())
#     return response_list
# start_time = time.time()
# train_df['response_text'] = train_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
# print("minutes to complete:",(time.time()-start_time)/60)
# train_df.head(10)

In [31]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the rows wtih split responses that have length 1 or 2.
dev_df = dev_df[dev_df['split_response'].str.len() > 2]
len(dev_df)

2041945

In [44]:
start_time = time.time()
dev_df['response_text'] = dev_df['response_text'].astype(str).apply(replace_pronouns,args=(gen_pronouns,))
print("minutes to complete:",(time.time()-start_time)/60)
dev_df.head(10)

minutes to complete: 0.5465651472409566


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,1965080,86681682,M,29909,James,thanks for passing this ridiculous legislation. https://www.youtube.com/watch?v=-y4wd8royre,Jim Banks,Congress_Republican,"[thanks, for, passing, this, ridiculous, legislation., https://www.youtube.com/watch?v=-y4wd8royre]"
1,1965081,86681682,M,29909,Martha,handsome young man on the left. 😂 mom,Jim Banks,Congress_Republican,"[handsome, young, man, on, the, left., 😂, mom]"
2,1965082,86681682,M,29911,Preston,you are sideways!,Jim Banks,Congress_Republican,"[you, are, sideways!]"
3,1965083,86681682,M,29911,Bobby,hi pronoun banks,Jim Banks,Congress_Republican,"[hi, congressman, banks]"
4,1965084,86681682,M,29911,Landon,"jim, i love your office. the 5th floor is for winners.",Jim Banks,Congress_Republican,"[jim,, i, love, your, office., the, 5th, floor, is, for, winners.]"
6,1965086,86681682,M,29911,Sean,it has the penthouse.,Jim Banks,Congress_Republican,"[it, has, the, penthouse.]"
7,1965087,86681682,M,29911,Preston,"we are thankful for you, mr. banks!",Jim Banks,Congress_Republican,"[we, are, thankful, for, you,, mr., banks!]"
10,1965090,86681682,M,29911,Justin,looks like lots of space for activities.,Jim Banks,Congress_Republican,"[looks, like, lots, of, space, for, activities.]"
12,1965092,86681682,M,29911,Bobby,my friend tanner,Jim Banks,Congress_Republican,"[my, friend, tanner]"
13,1965093,86681682,M,29911,William,"nice office, now when are you going to call for an investigation into the possible treasonable o...",Jim Banks,Congress_Republican,"[nice, office,, now, when, are, you, going, to, call, for, an, investigation, into, the, possibl..."


In [0]:
# Remove the columns of lists.
train_df = train_df.drop(['split_response'], axis=1)
dev_df = dev_df.drop(['split_response'], axis=1)


In [46]:
import gc
gc.collect()

695

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [49]:
train_df.op_gender.value_counts()

M    2900753
W    1061531
Name: op_gender, dtype: int64

In [50]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [0]:
# Set the y values.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
# Turn y values into 0 (female) and 1 (male).
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
# Convert y values to numpy arrays.
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [55]:
# Tokenize the sentences.
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 41 seconds


In [0]:
# Pad the tokenized data so all sequences are the same length; see first cell for max length.
# Length should be the same as in shortened response length above...
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [58]:
# Get the glove embeddings.
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
\ [4 files][  2.1 GiB/  2.1 GiB]   21.5 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
/
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [61]:
embedding_matrix.shape

(597881, 100)

In [62]:
# Initialize the model.
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           59788100  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [66]:
# Run the model.
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=20,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3962284 samples, validate on 2000000 samples
Epoch 1/20
3962284/3962284 [==============================] - 117s 29us/sample - loss: 0.5187 - acc: 0.7639 - val_loss: 0.4443 - val_acc: 0.8444
Epoch 2/20
3962284/3962284 [==============================] - 115s 29us/sample - loss: 0.5029 - acc: 0.7713 - val_loss: 0.4404 - val_acc: 0.8455
Epoch 3/20
3962284/3962284 [==============================] - 116s 29us/sample - loss: 0.4978 - acc: 0.7734 - val_loss: 0.4410 - val_acc: 0.8417
Epoch 4/20
3962284/3962284 [==============================] - 116s 29us/sample - loss: 0.4947 - acc: 0.7748 - val_loss: 0.4497 - val_acc: 0.8410
Epoch 5/20
3962284/3962284 [==============================] - 115s 29us/sample - loss: 0.4925 - acc: 0.7757 - val_loss: 0.4476 - val_acc: 0.8390
Epoch 6/20
3962284/3962284 [==============================] - 116s 29us/sample - loss: 0.4907 - acc: 0.7764 - val_loss: 0.4439 - val_acc: 0.8403
Epoch 7/20
3962284/3962284 [==============================] - 115s 29us/samp

In [67]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7823
Testing Accuracy:  0.8379


In [68]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6115826865950602


In [0]:
# can't get this to work
# plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [74]:
wrong_preds.op_gender.value_counts()

W    287622
M     43400
Name: op_gender, dtype: int64

In [75]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.03253515643173543

In [76]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2060436,11200190,48870736,W,394386,Grant,"aca raised my health care from $600 to $1,600 per month. it is anything but affordable. betty wi...",Betty McCollum,Congress_Democratic,0.529842,M
1983886,11033278,86569077,W,389881,Deanna,i like the idea of paid fml but i would much rather see employers raise employees income overall...,Kirsten Gillibrand,Congress_Democratic,0.700526,M
2022583,11071975,86569077,W,390238,Matt,"thank you kirsten, running after the latest headlines !",Kirsten Gillibrand,Congress_Democratic,0.560060,M
1924717,10974109,86569077,W,389341,Raelee,thank you so much for committing to voting against betsy devos.,Kirsten Gillibrand,Congress_Democratic,0.631508,M
2037193,11086585,86569077,W,390472,Dwight,"mmmm lets see here donna karan, fashion designer and the creator of the donna karan new york and...",Kirsten Gillibrand,Congress_Democratic,0.791945,M
2037981,11087373,86569077,W,390484,Ron,"i have no problem with ending the subsidies, but i also do not mind paying more for all goods. g...",Kirsten Gillibrand,Congress_Democratic,0.734650,M
2113460,11254440,77234050,W,399626,Debra,cecile charles i posted the table for you - it is for wa state,Cathy McMorris Rodgers,Congress_Republican,0.678835,M
2153417,11294397,77234050,W,400418,Hugh,former marine thanks you.,Cathy McMorris Rodgers,Congress_Republican,0.770747,M
2124542,11265522,77234050,W,399735,Robi,keep fighting the good fight,Cathy McMorris Rodgers,Congress_Republican,0.747467,M
432281,3365516,76269753,W,103764,Eddie,automatic weapons are already controlled under the nfa.,Dianne Feinstein,Congress_Democratic,0.871397,M


In [77]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2067033,11206787,48870736,W,395135,Michele,thank you pronoun mccollum for standing up for immigration law reform.,Betty McCollum,Congress_Democratic,0.999967,M
1993131,11042523,86569077,W,389977,Colleen,you mean mr paul mark? rand opportunist paul?,Kirsten Gillibrand,Congress_Democratic,0.999907,M
2126233,11267213,77234050,W,399739,Marilyn,i see nothing wrong with what rand and mitch were discussing. you do not thin the dems are not d...,Cathy McMorris Rodgers,Congress_Republican,0.999891,M
2061378,11201132,48870736,W,394429,Patrick,"thank you for your good work, pronoun mccollum",Betty McCollum,Congress_Democratic,0.999882,M
2061353,11201107,48870736,W,394425,Kate,"thank you, rep. mccollum, for doing everything you can to stand up against corruption.",Betty McCollum,Congress_Democratic,0.999869,M
2045576,11094968,86569077,W,390593,Danielle,@ duncan - i benefitted because i realized i disagree with rand's view of the world. thanks for ...,Kirsten Gillibrand,Congress_Democratic,0.999868,M
1936262,10985654,86569077,W,389409,Colleen,"your either mentally ill or you dont care for the rule of law...she is a liar, pronoun is corrup...",Kirsten Gillibrand,Congress_Democratic,0.999858,M
2061983,11201737,48870736,W,394492,Kimberly,"excellent, thank you pronoun mccollum ! keep up the good fight -we need your voice in dc.",Betty McCollum,Congress_Democratic,0.999850,M
387717,3137808,23984618,W,83521,Sebastian,the original congress was all-volunteer with no pay. perhaps mr. moran should be reminded how tr...,Ann Wagner,Congress_Republican,0.999848,M
2067097,11206851,48870736,W,395153,Peggy,"miigwech, pronoun mccollum!",Betty McCollum,Congress_Democratic,0.999838,M


In [78]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1806560,10329697,32570461,M,382375,Dave,angus king is a former governor of the state of maine. did you also make friends with former new...,Tim Kaine,Congress_Democratic,0.000025,W
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more suppo...",Marco Rubio,Congress_Republican,0.000025,W
1373869,7792163,99469231,M,296202,Michael,as the terrorist attacks abroad continue....here at home (ignorance) on the behalf of state leve...,John McCain,Congress_Republican,0.000029,W
1132701,5381755,46506154,M,162152,Laura,congressional candidate frank guinta did this in nh!,Marco Rubio,Congress_Republican,0.000046,W
1887369,10920297,10458808,M,386070,Stephen,yes yes yes. kamala. you have always been there for us!!! fantastic!,Mark Takano,Congress_Democratic,0.000052,W
181521,2358273,62246450,M,52502,Annie,keep going elizabeth! we support you!,Ron Wyden,Congress_Democratic,0.000055,W
653735,4902789,46506154,M,160224,Chris,"trump has actually been relatively evenhanded in doling out cash to the two parties, but since 1...",Marco Rubio,Congress_Republican,0.000084,W
477131,3752147,89414884,M,127106,Mike,"well, if i were the one in charge I had cut funding on everything that government has no busines...",Pete Visclosky,Congress_Democratic,0.000122,W
991706,5240760,46506154,M,161317,Christopher,send the senator a quick letter of support and to say thank you: http://www.ayotte.senate.gov/?p...,Marco Rubio,Congress_Republican,0.000137,W
1861625,10894553,10458808,M,385773,Kyle,rep. debbie dingell,Mark Takano,Congress_Democratic,0.000172,W
